## Initialize

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

In [ ]:
data_results_path

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "211007"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))

predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics.feather")) 
loghazards = arrow::read_feather(glue("{data_results_path}/loghazards_model_{run}_metabolomics.feather")) %>% pivot_longer(starts_with("logh"), names_to=c("endpoint", "features"), values_to="logh", names_pattern="logh_?(.*)_(.*)$")

In [ ]:
colnames(predictions)

## Attributions by correlation

In [ ]:
library(ggforestplot)

In [ ]:
# Deepexplainer
attributions_de = arrow::read_feather(glue("{data_results_path}/attributions_211026.feather")) %>% mutate(explainer="DeepExplainer")

In [ ]:
attributions = attributions_de#bind_rows(attributions_de, attributions_ke)

## Attributions by shap

In [ ]:
library("ggbeeswarm")

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer"
                   
)

In [ ]:
library(ggforestplot)
ng_names = df_NG_biomarker_metadata %>% mutate(metabolite = str_replace_all(tolower(description), " ", "_"))
ng_names %>% sample_n(10)

In [ ]:
ng_names %>% select(group, subgroup) %>% distinct() %>% arrange(group, subgroup)

In [ ]:
library(fuzzyjoin)

In [ ]:
subgroup_order = c( 'Amino acids',
                    'Branched-chain amino acids',
                   'Aromatic amino acids',
                   'Fluid balance',
                   'Inflammation',
                    'Fatty acids',
                    'Glycolysis related metabolites',
                    'Ketone bodies',
         
                   'Total lipids',
                    'Cholesterol',
                    'Free cholesterol',
                   'Cholesteryl esters',
                   'Phospholipids',
                   'Triglycerides',
                   'Other lipids',
                   
                    'Lipoprotein particle sizes',
                    'Lipoprotein particle concentrations',
                    'Chylomicrons and extremely large VLDL',
                   'Very large VLDL',
                   'Large VLDL',
                   'Medium VLDL',
                   'Small VLDL',
                   'Very small VLDL',
              
                   'Large LDL',
                   'Medium LDL',
                   'Small LDL',
                    'IDL',
                   'Very large HDL',
                   'Large HDL',
                   'Medium HDL',
                   'Small HDL',
                   'Apolipoproteins'
                  )

In [ ]:
mets1 = attributions %>% select(metabolite) %>% distinct() %>% left_join(ng_names, by = "metabolite")
mets2 = mets1 %>% filter(is.na(name)) %>% select(metabolite) %>% stringdist_left_join(ng_names, by = "metabolite", max_dist = 1) %>% 
    rename(metabolite = metabolite.x) %>% select(-metabolite.y) %>% distinct()
mets3 = mets2 %>% filter(is.na(name)) %>% select(metabolite) %>% stringdist_left_join(ng_names, by = "metabolite", max_dist = 8) %>% 
    rename(metabolite = metabolite.x) %>% select(-metabolite.y) %>% distinct() 
mets = bind_rows(mets1 %>% filter(!is.na(name)), mets2 %>% filter(!is.na(name)), mets3) %>% mutate(subgroup = factor(subgroup, levels=subgroup_order)) %>% arrange(subgroup, abbreviation)
abbreviation_order = mets$abbreviation
metabolite_order = mets$metabolite
mets %>% sample_n(5)

In [ ]:
attributions_metadata = attributions %>% left_join(mets %>% select(metabolite, abbreviation, group, subgroup), by="metabolite") %>% mutate(eid=as.integer(as.character(eid)))

In [ ]:
library(gghighlight)

## Global attributions

In [ ]:
attrib_raw = arrow::read_feather(glue("{data_results_path}/REVISION_220110_attrib_raw.feather"))

In [ ]:
attrib_sample = attrib_raw %>% group_by(endpoint, metabolite, explainer) %>% 
    mutate(shap_quantile=ntile(shap, 100), met_quantile=ntile(met_value, 100)) #%>% 

In [ ]:
library(ggforce)

## UMAP

In [ ]:
temp_umap_shap = arrow::read_feather(glue("{data_results_path}/REVISION_220111_temp_umap_shap.feather"))

In [ ]:
endpoint="M_type_2_diabetes"

In [ ]:
temp_ep = temp_umap_shap %>% filter(endpoint==!!endpoint) %>% filter(x>2&x<12) %>% filter(hr>5)#&hr<12) #'%>% filter(event==FALSE)

In [ ]:
tempA = temp_ep %>% filter(eid == 2475864)
tempB = temp_ep %>% filter(eid == 1239178)
tempC = temp_ep %>% filter(eid == 1690024)

In [ ]:
calc_dist = function(xref, yref, x, y){sqrt((x-xref)^2+(y-yref)^2)}
temp_areas = temp_ep %>% mutate(distA = calc_dist(tempA$x, tempA$y, x, y), distB = calc_dist(tempB$x, tempB$y, x, y), distC = calc_dist(tempC$x, tempC$y, x, y)) %>%
    pivot_longer(c(starts_with("dist"), -contains("disturbance")), names_to="reference", values_to="dist", names_prefix="dist") %>%
    mutate(area = case_when(reference=="A"&dist<1 ~ "A", reference=="B"&dist<1 ~ "B", reference=="C"&dist<1 ~ "C", dist>=1 ~ "None")) %>% select(-dist, -reference) %>% distinct()

In [ ]:
temp_plot = temp_areas %>% filter(area != "None") %>%
    select(eid, x, y, endpoint, hr, event, incident_event, age_at_recruitment, sex, area, starts_with("NMR_"), 
           -NMR_measurement_quality_flagged, -NMR_spectrometer, -NMR_FLAG) %>% 
    pivot_longer(c(starts_with("NMR_")), names_to="metabolite", values_to="value") %>%
    mutate(metabolite=str_remove_all(metabolite, "NMR_")) %>% 
    left_join(mets %>% select(metabolite, abbreviation, subgroup)) %>% 
    mutate(subgroup = factor(subgroup, levels=subgroup_order)) %>% 
    mutate(metabolite = factor(metabolite, levels=metabolite_order)) %>% 
    mutate(abbreviation = factor(abbreviation, levels=abbreviation_order))

In [ ]:
library(ggdist)

In [ ]:
base_size = 10
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=10, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2)))

In [ ]:
plot_width=17.5; plot_height=22; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)


met_diffs = ggplot(temp_plot, aes(x=factor(area, levels=c("A", "B", "C")), y=value)) +
    labs(x=NULL, y="Metabolite value")+
    geom_violin(alpha=0.7, size=0.25, color="#023768", fill="#023768")+
    facet_wrap(~abbreviation, scales="free", ncol=10)+
    theme(legend.position="none") + theme(strip.text.x = element_text(size = 11))

met_diffs

In [ ]:
library(gt)
plot_name = "Suppl_Figures_9_UMAPAreas"
met_diffs %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device="pdf", width=plot_width, height=plot_height, dpi=plot_dpi)